In [1]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    num_words=10000
)
word_index = reuters.get_word_index()


2021-12-11 11:47:14.264926: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-11 11:47:14.264983: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
len(train_data)


8982

In [3]:
print(len(test_data))
print(type(test_data))
test_data.ndim


2246
<class 'numpy.ndarray'>


1

In [4]:
print(len(word_index))
print(type(word_index))


30979
<class 'dict'>


In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


In [6]:
decoded_news = " ".join([reverse_word_index.get(i - 3, "?") for i in train_data[0]])
decoded_news


'? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

In [7]:
import numpy as np


def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.0
    return results


In [8]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)


In [9]:
np.unique(train_labels)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])

In [10]:
def to_one_hot(labels, dimmension=46):
    results = np.zeros((len(labels), dimmension))
    for i, label in enumerate(labels):
        results[i, label] = 1.0
    return results


# bu fonksiyon ile aynı şeyi yapan şey:
# from keras.utils.np_utils import to_categorical


In [11]:
one_hot_train_labels = to_one_hot(train_labels)
one_hot_train_labels


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
one_hot_test_labels = to_one_hot(test_labels)
one_hot_test_labels


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
from keras import models, layers


In [14]:
model = models.Sequential()


2021-12-11 11:47:28.989714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-11 11:47:28.989813: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-11 11:47:28.989880: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pc1): /proc/driver/nvidia/version does not exist
2021-12-11 11:47:29.021328: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
model.add(layers.Dense(64, activation="relu", input_shape=(10000,)))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(46, activation="softmax"))


In [16]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)


In [17]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]


In [18]:
history = model.fit(
    x=partial_x_train,
    y=partial_y_train,
    batch_size=512,
    epochs=2,
    validation_data=(x_val, y_val),
)


2021-12-11 11:57:35.283261: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 319280000 exceeds 10% of free system memory.


Epoch 1/2
15/16 [===========================>..] - ETA: 0s - loss: 2.7575 - accuracy: 0.5111

2021-12-11 11:58:02.842615: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 40000000 exceeds 10% of free system memory.


16/16 [==============================] - 19s 136ms/step - loss: 2.7224 - accuracy: 0.5167 - val_loss: 1.7702 - val_accuracy: 0.6410
Epoch 2/2
16/16 [==============================] - 1s 44ms/step - loss: 1.4325 - accuracy: 0.7035 - val_loss: 1.3386 - val_accuracy: 0.7060
